In [155]:
# import libraries, packages
import numpy as np
import matplotlib.pyplot as plt
import scipy
import math
import scipy.optimize
import pandas as pd
import random

In [253]:
# USER PARAMETERS ADJUST BEFORE USING
# lender's beliefs about group probabilities of repayment
pi_w = 0.5 
pi_b = 0.3

#distribution of creditworthy individuals in b/w
b_credit = 0.25
w_credit = 0.5

# benefit of getting a loan to a borrower
benefit = 10
# x_q = 5 # net return to lender if qualified borrower gets loan, q = qualified
# x_u = -10 # net loss to lender if unqualified borrower gets loan, u = unqualified
# ratio = abs(x_q / x_u) # ratio of net gain to loss

ratio = 100

# change in lender's perceptions? (how creditworthiness improves)
alpha = 0.05

# INCOMPLETE cost to become creditworthy. Cost varies for each member but is the same distribution for the both groups
cost = 0

# INCOMPLETE fraction of workers that choose to become creditworthy G(c)


In [230]:
# Borrower: each borrower has a group membership (and group creditworthiness signal), individual signal (theta), and actual credtiworthiness
# true if group w, false if group b
# theta = borrower noisy signal
# pi = group signal
class Borrower:
  def __init__(self, group, theta, is_qualified, pi = None):
    self._group = group
    if group:
      self._pi = pi_w
    else:
      self._pi = pi_b      
    self._theta = theta
    self._is_qualified = is_qualified

  @property
  def group(self):
        return self._group
  @property
  def theta(self):
        return self._theta
  @property
  def pi(self):
        return self._pi
  @property
  def is_qualified(self):
      return self._is_qualified
  
  def __str__(self):
        name = "w" if self.group else "b"

        return f"Group: {name}, is_qualified: {self._is_qualified * 1}, theta: {self.theta}, pi: {self.pi}"
  
# Intialize simulation distributions


# create whatever number of objects
# create the actual signals based on the proportion, then create a noisy signal theta
# FLAG LETS TALK ABOUT NOISE
  # def __init__(self, group, theta, is_qualified, pi = None):

def create_borrowers(group, qualified_percentage, total_objects=100):
    borrowers = []
    qualified_count = int(total_objects * qualified_percentage)
    for _ in range(qualified_count):
        theta = 1 - abs(random.gauss(0, 0.25))
        theta = max(0, min(1, theta))
        borrowers.append(Borrower(group, theta, True))
    
    for _ in range(total_objects - qualified_count):
        theta = abs(random.gauss(0, 0.25))
        theta = max(0, min(1, theta))
        borrowers.append(Borrower(group, theta, False))
    
    return borrowers

# groups of borrowers for simulation
b = create_borrowers(False, b_credit)
w = create_borrowers(True, w_credit)

q = [borrower for borrower in b+w if borrower.is_qualified]
u = [borrower for borrower in b+w if not borrower.is_qualified]

for x in b+w:
    print(x)


Group: b, is_qualified: 1, theta: 0.6385800214244146, pi: 0.3
Group: b, is_qualified: 1, theta: 0.6908767916350265, pi: 0.3
Group: b, is_qualified: 1, theta: 0.9124334061154983, pi: 0.3
Group: b, is_qualified: 1, theta: 0.9128031144879696, pi: 0.3
Group: b, is_qualified: 1, theta: 0.5385724813240016, pi: 0.3
Group: b, is_qualified: 1, theta: 0.8135755555435211, pi: 0.3
Group: b, is_qualified: 1, theta: 0.6540128290441632, pi: 0.3
Group: b, is_qualified: 1, theta: 0.7030161401530166, pi: 0.3
Group: b, is_qualified: 1, theta: 0.878260766619755, pi: 0.3
Group: b, is_qualified: 1, theta: 0.8625600726544094, pi: 0.3
Group: b, is_qualified: 1, theta: 0.9198366577066012, pi: 0.3
Group: b, is_qualified: 1, theta: 0.8765484580697612, pi: 0.3
Group: b, is_qualified: 1, theta: 0.7475504408890351, pi: 0.3
Group: b, is_qualified: 1, theta: 0.9900859131779018, pi: 0.3
Group: b, is_qualified: 1, theta: 0.8931434803562366, pi: 0.3
Group: b, is_qualified: 1, theta: 0.49203851346196037, pi: 0.3
Group: b

Functions

In [249]:

# probability signal does not exceed theta in F_q(theta)
def signal_probability(set, given_theta):
    if not 0 <= given_theta <= 1:
        raise ValueError("Threshold must be between 0 and 1")
    count = sum(1 for individual in set if individual.theta <= given_theta)
    return count / len(set) if set else 0.0

# density functions for signal probability, helper function to generate the pdf
# should theere be rounding in this function?
def generate_pdf(group):
    total_borrowers = len(group)
    if total_borrowers == 0:
        raise ValueError("The borrowers list is empty")
    
    theta_counts = {}
    for borrower in group:
        theta = round(borrower.theta, 1)
        if theta in theta_counts:
            theta_counts[theta] += 1
        else:
            theta_counts[theta] = 1

    # Convert counts to probabilities
    pdf = {theta: count / total_borrowers for theta, count in theta_counts.items()}
    return pdf

# gives the specific pdf value of a group given theta
def signal_pdf(group, given_theta):
    if not 0 <= given_theta <= 1:
        raise ValueError("given_theta must be between 0 and 1")

    # Generate the PDF
    pdf = generate_pdf(group)

    # Sum the probabilities for theta values less than x
    sum_prob = sum(prob for theta, prob in pdf.items() if theta < given_theta)

    return sum_prob

# liklihood ratio at a given theta
def likelihood_ratio(given_theta):
    q_value = signal_pdf(q, given_theta)

    if q_value == 0:
        return 1

    return signal_pdf(u, given_theta) / q_value
    
# Lender's posterior probability borrower is qualified
def qualified_prob(borrower):
    pi = borrower.pi
    if pi == 0:
            raise ValueError("pi should not be zero.")
    if pi == 1:
            raise ValueError("pi should not be one.")
    return 1 / (1+(((1-pi)/pi) * likelihood_ratio(borrower.theta)))

# Lender's expected payoff from lending to anyone
def payoff(borrower):
    return qualified_prob(borrower) * x_q - (1-qualified_prob(borrower)) * x_u


#lender lends iff
def employer_assignment(borrower):
    return ratio >= ((1-borrower.pi)/(borrower.pi))*likelihood_ratio(borrower.theta)


# how employer determines the standard s*
# minimum value of theta such that r > [(1-pi/pi)*liklihood]
# the standard is set differently per group
def standard(group, theta_bounds=(0, 1), tol=1e-6):
    if group:
        pi = pi_w
    else:
        pi = pi_b
  # The value we are comparing against
    compare_value = (1 - pi) / pi
    print("compare value")
    print(compare_value)

    # Define the function for the root finding
    def inequality(theta):
        return ratio - compare_value * likelihood_ratio(theta)
    
    # FLAG if there's no solution, then the lender lends to no one, and the credit standard is `100%`
    if np.sign(ratio - compare_value * likelihood_ratio(0)) == np.sign(ratio - compare_value * likelihood_ratio(1)):
        print("MEGA FLAG")
        print(ratio - compare_value * likelihood_ratio(0))
        print(ratio - compare_value * likelihood_ratio(1))
        return 1

    # Use a numerical solver to find the root within the given bounds
    result = scipy.optimize.root_scalar(inequality, bracket=theta_bounds, method='brentq', xtol=tol)

    if not result.converged:
        raise ValueError("Solver did not converge to a solution")

    return result.root
    

In [254]:
# issue is the solver function, giving wrong standard values

signal_probability(b, 0.5)
generate_pdf(b)
def simulate():
    standard_b = standard(False)
    standard_w = standard(True)

    return standard_b, standard_w

print(simulate())

compare value
2.3333333333333335
MEGA FLAG
97.66666666666667
97.22222222222223
compare value
1.0
MEGA FLAG
99.0
98.80952380952381
(1, 1)


In [247]:
values_list = [i / 100.0 for i in range(101)]

for x in values_list:
    print(ratio - 1 * likelihood_ratio(x))

2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
-67.8
-67.8
-67.8
-67.8
-67.8
-67.8
-67.8
-67.8
-67.8
-67.8
-7.457142857142857
-7.457142857142857
-7.457142857142857
-7.457142857142857
-7.457142857142857
-7.457142857142857
-7.457142857142857
-7.457142857142857
-7.457142857142857
-7.457142857142857
-4.38
-4.38
-4.38
-4.38
-4.38
-4.38
-4.38
-4.38
-4.38
-4.38
-1.1333333333333329
-1.1333333333333329
-1.1333333333333329
-1.1333333333333329
-1.1333333333333329
-1.1333333333333329
-1.1333333333333329
-1.1333333333333329
-1.1333333333333329
-1.1333333333333329
0.8742857142857146
0.8742857142857146
0.8742857142857146
0.8742857142857146
0.8742857142857146
0.8742857142857146
0.8742857142857146
0.8742857142857146
0.8742857142857146
0.8742857142857146
1.8095238095238095
1.8095238095238095
1.8095238095238095
1.8095238095238095
1.8095238095238095
1.8095238095238095
1.8095238095238095
1.8

In [ ]:


# Simulation element
def simulate_borrowers(borrowers, q, u, num_iterations=1000):
    results = []
    for _ in range(num_iterations):
        for borrower in borrowers:
            standard_value = standard(q, u, borrower.pi, ratio)
            assignment_prob = prob_assignment_given_group([borrower], standard_value)
            net_payoff_value = net_payoff(borrower, q, u, cost)
            results.append({
                'group': borrower.group,
                'pi': borrower.pi,
                'theta': borrower.theta,
                'standard': standard_value,
                'assignment_prob': assignment_prob,
                'net_payoff': net_payoff_value
            })
    return results
# Adjusting credit rating
def update_credit_rating(group, increase):
    for borrower in group:
        borrower.pi += increase
        borrower.pi = min(borrower.pi, 1)  # Ensure pi does not exceed 1
# Run the simulation
results = simulate_borrowers(tester_q + tester_u, tester_q, tester_u, num_iterations=1000)
# Print a summary of results
results_df = pd.DataFrame(results)
print(results_df.describe())

within linkelihood function, q_value
0
within linkelihood function, q_value
0.6666666666666666
within linkelihood function, q_value
0
within linkelihood function, q_value
0.6666666666666666
within linkelihood function, q_value
0
likelihood ratio: 
1
term 2
5.0
within linkelihood function, q_value
0
likelihood ratio: 
1
term 2
5.0
within linkelihood function, q_value
0
within linkelihood function, q_value
0.6666666666666666
within linkelihood function, q_value
0
within linkelihood function, q_value
0.6666666666666666
within linkelihood function, q_value
0.3333333333333333
likelihood ratio: 
3.0
term 2
5.0
within linkelihood function, q_value
0.3333333333333333
likelihood ratio: 
3.0
term 2
5.0
within linkelihood function, q_value
0
within linkelihood function, q_value
0.6666666666666666
within linkelihood function, q_value
0
within linkelihood function, q_value
0.6666666666666666
within linkelihood function, q_value
0.6666666666666666
likelihood ratio: 
1.5
term 2
5.0
within linkelihood